In [25]:
from dotenv import load_dotenv

import numpy as np
import pandas as pd
import os, boto3, io


load_dotenv()
BUCKET = os.getenv("BUCKET")
PROCESSED_DATA_FOLDER = os.getenv("PROCESSED_DATA_FOLDER")
PATCH = "patch_15_6"
GRANULARITY_LIST = ["single_user", "champion_x_role", "champion_x_role_x_user"]
AWS_KEY = os.getenv("AWS_KEY")
AWS_SECRET = os.getenv("AWS_SECRET")

In [26]:
# Test your credentials
print(f"AWS_KEY loaded: {'Yes' if AWS_KEY else 'No'}")
print(f"AWS_SECRET loaded: {'Yes' if AWS_SECRET else 'No'}")

# Test S3 connection
try:
    s3 = boto3.client(
        "s3",
        aws_access_key_id=AWS_KEY,
        aws_secret_access_key=AWS_SECRET,
        region_name="us-east-2"
    )
    response = s3.list_buckets()
    print("✅ AWS credentials working!")
    print(f"Available buckets: {[bucket['Name'] for bucket in response['Buckets']]}")
except Exception as e:
    print(f"❌ AWS credentials error: {e}")

AWS_KEY loaded: Yes
AWS_SECRET loaded: Yes
✅ AWS credentials working!
Available buckets: ['lolhelper', 'sagemaker-us-east-2-550149510806']


In [33]:
# Code in case we want to pull all dfs in a single function

def get_processed_data_file(granularity: str, role=None) -> pd.DataFrame:

    if granularity == "single_user":
        key = f"{PROCESSED_DATA_FOLDER}/single_user_data/{PATCH}/single_user_aggregated_data.csv"
    elif granularity == "champion_x_role":
        key = f"{PROCESSED_DATA_FOLDER}/champion_x_role/{PATCH}/champion_x_role_aggregated_data.csv"
    elif granularity == "champion_x_role_x_user":
        key = f"{PROCESSED_DATA_FOLDER}/champion_x_role_x_user/{PATCH}/champion_x_role_x_user_aggregated_data.csv"
    elif granularity == "cluster":
        key = f"{PROCESSED_DATA_FOLDER}/clusters/{PATCH}/{role}_vectors_df.csv"

    else:
        raise ValueError("Incorrect granularity input, must be 'champion_x_role', 'champion_x_role_x_user', 'single_user', or 'cluster'")
    
    # Pull the object
    s3 = boto3.client(
        "s3",
        aws_access_key_id=AWS_KEY,
        aws_secret_access_key=AWS_SECRET,
        region_name="us-east-2"
    )

    print(key)
    obj = s3.get_object(Bucket=BUCKET, Key=key)

    # Read it straight into pandas
    df = pd.read_csv(io.BytesIO(obj["Body"].read()))

    print(df.shape)

    return df


In [ ]:
champ_x_role_df = get_processed_data_file("champion_x_role")
display(champ_x_role_df.head())


processed_data/champion_x_role/patch_15_6/champion_x_role_aggregated_data.csv
(834, 267)


,champion_id,champion_name,team_position,total_games_played_in_role,total_games_per_champion,avg_kills,avg_deaths,avg_deaths_by_enemy_champs,avg_assists,avg_kill_participation,...,pct_of_matches_with_vision,pct_of_matches_with_spell_vamp,pct_of_matches_with_boots,pct_of_matches_with_aura,pct_of_matches_with_magic_penetration,pct_of_matches_with_cooldown_reduction,pct_of_matches_with_armor,pct_of_matches_with_health_regen,pct_of_matches_with_mana,pct_of_matches_with_nonboots_movement
0,26,Zilean,MIDDLE,716,5017,2.766760,3.621508,3.603352,10.705307,0.480971,...,2.444545,0.000000,4.708013,0.769579,1.177003,21.004980,1.629697,3.123585,36.441829,27.885921
1,26,Zilean,UTILITY,4116,5017,1.772595,4.638241,4.614674,13.801749,0.560299,...,39.115874,0.000000,5.953710,6.839324,0.327454,41.110367,9.064523,32.693310,7.070031,22.147801
2,26,Zilean,BOTTOM,38,5017,2.078947,4.657895,4.657895,9.421053,0.469747,...,4.424779,0.000000,4.424779,0.000000,1.769912,20.353982,2.654867,4.424779,29.203540,23.008850
3,26,Zilean,TOP,147,5017,2.605442,4.047619,4.013605,10.088435,0.439549,...,3.037383,1.168224,3.738318,3.037383,1.635514,19.158879,4.906542,7.476636,41.121495,21.962617
4,29,Twitch,BOTTOM,7638,8133,8.688662,6.275072,6.256612,5.903509,0.505619,...,0.594014,15.966035,0.175158,0.000000,0.396009,0.483588,0.658746,0.742518,0.019039,21.593938


In [29]:
sngle_user_df = get_processed_data_file("single_user")
display(sngle_user_df.head())

processed_data/single_user_data/patch_15_6/single_user_aggregated_data.csv
(8, 265)


,champion_id,champion_name,team_position,total_games_played_in_role,total_games_per_champion,avg_kills,avg_deaths,avg_deaths_by_enemy_champs,avg_assists,avg_kill_participation,...,pct_of_matches_with_vision,pct_of_matches_with_spell_vamp,pct_of_matches_with_boots,pct_of_matches_with_aura,pct_of_matches_with_magic_penetration,pct_of_matches_with_cooldown_reduction,pct_of_matches_with_armor,pct_of_matches_with_health_regen,pct_of_matches_with_mana,pct_of_matches_with_nonboots_movement
0,12,Alistar,UTILITY,1,1,0.000000,0.000000,0.000000,0.000000,NaN,...,50.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,50.000000,0.0,0.000000
1,54,Malphite,TOP,1,1,3.000000,1.000000,1.000000,3.000000,0.206897,...,0.000000,0.000000,0.000000,0.0,0.0,50.000000,100.000000,0.000000,0.0,0.000000
2,84,Akali,TOP,1,1,1.000000,5.000000,5.000000,1.000000,0.181818,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,92,Riven,TOP,71,76,6.380282,5.802817,5.676056,5.028169,0.392077,...,2.118644,19.067797,1.694915,0.0,0.0,63.559322,4.237288,4.661017,0.0,3.813559
4,92,Riven,MIDDLE,5,76,9.400000,3.600000,3.600000,5.600000,0.515483,...,0.000000,18.750000,0.000000,0.0,0.0,75.000000,6.250000,0.000000,0.0,0.000000


In [36]:
pd.set_option('display.max_columns', None)

In [38]:
top_cluster_df = get_processed_data_file("cluster", "top")
display(top_cluster_df.head())

processed_data/clusters/patch_15_6/top_vectors_df.csv
(61, 113)


,champion_name,team_position,cluster,euclidean_distance_to_centroid,residual_vec_scaled,pct_of_matches_with_ghost_x,avg_individual_solo_towers_kills_late_game_x,avg_ability_uses_x,pct_of_games_first_blood_kill_x,avg_max_level_lead_over_lane_opp_x,avg_skillshots_dodged_x,average_damage_per_minute_x,avg_times_immobilize_and_kill_with_ally_x,avg_kills_near_enemy_turret_x,avg_number_of_multikills_x,avg_assists_x,avg_first_takedown_time_x,avg_solo_kills_x,avg_times_pick_kill_with_ally_x,avg_initial_buff_count_x,avg_skillshots_landed_early_game_x,avg_quick_solo_kills_x,avg_individual_inhibitor_kills_x,role_popularity_for_champion_x,pct_of_games_with_early_lane_phase_gold_exp_adv_x,avg_skillshots_hit_x,avg_outnumbered_kills_x,total_games_fastest_item_completion_x,avg_times_applied_cc_on_others_x,avg_deaths_x,avg_enemy_champion_immobilizations_x,avg_number_of_items_purchased_x,avg_damage_self_mitigated_x,avg_time_ccing_others_x,pct_of_games_indiv_killed_1st_tower_x,pct_of_matches_with_ignite_x,avg_damage_taken_per_death_x,avg_kill_participation_x,avg_gold_per_minute_x,avg_individual_void_monster_kills_x,pct_of_games_with_lanephase_gold_exp_adv_x,avg_kills_x,avg_individual_tower_assists_x,pct_true_damage_x,avg_minions_killed_x,avg_kills_on_other_lanes_early_as_laner_x,avg_times_survived_three_immobilizes_in_fight_x,avg_killing_sprees_x,avg_damage_dealt_to_champions_x,pct_of_matches_with_teleport_x,avg_gold_earned_per_game_x,avg_control_ward_time_coverage_in_river_or_enemy_half_x,avg_indiv_dmg_dealt_to_buildings_x,avg_cs_per_minute_x,avg_time_played_per_game_minutes_x,avg_vision_score_advantage_over_lane_opponent_x,avg_total_healing_x,avg_effective_heal_and_shield_x,avg_indiv_turret_plates_taken_x,pct_of_matches_with_ghost_y,avg_individual_solo_towers_kills_late_game_y,avg_ability_uses_y,pct_of_games_first_blood_kill_y,avg_max_level_lead_over_lane_opp_y,avg_skillshots_dodged_y,average_damage_per_minute_y,avg_times_immobilize_and_kill_with_ally_y,avg_kills_near_enemy_turret_y,avg_number_of_multikills_y,avg_assists_y,avg_first_takedown_time_y,avg_solo_kills_y,avg_times_pick_kill_with_ally_y,avg_initial_buff_count_y,avg_skillshots_landed_early_game_y,avg_quick_solo_kills_y,avg_individual_inhibitor_kills_y,role_popularity_for_champion_y,pct_of_games_with_early_lane_phase_gold_exp_adv_y,avg_skillshots_hit_y,avg_outnumbered_kills_y,total_games_fastest_item_completion_y,avg_times_applied_cc_on_others_y,avg_deaths_y,avg_enemy_champion_immobilizations_y,avg_number_of_items_purchased_y,avg_damage_self_mitigated_y,avg_time_ccing_others_y,pct_of_games_indiv_killed_1st_tower_y,pct_of_matches_with_ignite_y,avg_damage_taken_per_death_y,avg_kill_participation_y,avg_gold_per_minute_y,avg_individual_void_monster_kills_y,pct_of_games_with_lanephase_gold_exp_adv_y,avg_kills_y,avg_individual_tower_assists_y,pct_true_damage_y,avg_minions_killed_y,avg_kills_on_other_lanes_early_as_laner_y,avg_times_survived_three_immobilizes_in_fight_y,avg_killing_sprees_y,avg_damage_dealt_to_champions_y,pct_of_matches_with_teleport_y,avg_gold_earned_per_game_y,avg_control_ward_time_coverage_in_river_or_enemy_half_y,avg_indiv_dmg_dealt_to_buildings_y,avg_cs_per_minute_y,avg_time_played_per_game_minutes_y,avg_vision_score_advantage_over_lane_opponent_y,avg_total_healing_y,avg_effective_heal_and_shield_y,avg_indiv_turret_plates_taken_y
0,Yone,TOP,8,3.319800,"[-0.3673, -0.5033, 1.0261, -0.3538, 0.5449, -0...",-0.541732,-0.223908,1.046836,-0.349918,0.690293,-0.885159,0.841719,0.231230,0.360882,0.010413,-0.736694,-0.356868,0.676196,-0.825517,-0.277538,-0.062111,0.061101,-0.415061,-0.590416,-0.182609,-0.131632,0.602551,0.409129,-0.513699,1.278454,0.356791,-0.553498,-0.181853,-0.218666,-0.205050,-0.564557,-0.950534,-0.844827,0.346199,0.133519,0.310353,0.106695,-1.166264,0.584275,0.641346,-0.268061,-0.573390,0.038958,0.745793,0.532604,0.259971,0.281217,-0.485310,0.981879,-0.679351,-0.251025,-0.151198,-0.275905,-0.188745,-0.174388,0.279361,0.020691,0.003891,0.145392,-0.448346,0.329052